# RAG Pipeline Exploration

This notebook explores the CivicNav RAG pipeline components.

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

from app.models.schemas import Category, IntentClassification
from app.config import get_settings

settings = get_settings()
print(f"OpenAI Endpoint: {settings.azure_openai_endpoint}")
print(f"Search Endpoint: {settings.azure_search_endpoint}")

## 1. Intent Classification

The QueryAgent classifies user intent into categories.

In [ ]:
# Test intent classification
from app.agents.query_agent import QueryAgent

agent = QueryAgent()
result = await agent.execute("When is trash pickup?")

print(f"Category: {result.output.category}")
print(f"Confidence: {result.output.confidence}")
print(f"Entities: {result.output.entities}")

## 2. Hybrid Search

The RetrieveAgent combines vector, keyword, and semantic search.

In [ ]:
# Test search
from app.tools.search_tool import get_search_tool
from app.tools.openai_tool import get_openai_tool

openai_tool = get_openai_tool()
search_tool = get_search_tool()

# Create embedding
query = "trash collection schedule"
embedding = await openai_tool.create_embedding(query)

# Hybrid search
results = await search_tool.hybrid_search(query, embedding, top_k=3)

for r in results:
    print(f"- {r.title} (score: {r.relevance_score:.2f})")

## 3. Answer Synthesis

The AnswerAgent generates responses with citations.

In [ ]:
# Test full pipeline
from app.agents.query_agent import QueryAgent
from app.agents.retrieve_agent import RetrieveAgent
from app.agents.answer_agent import AnswerAgent

query = "How do I get a building permit?"

# Stage 1: Query
query_agent = QueryAgent()
intent_result = await query_agent.execute(query)
intent = intent_result.output

# Stage 2: Retrieve
retrieve_agent = RetrieveAgent()
search_result = await retrieve_agent.execute((query, intent))
results = search_result.output

# Stage 3: Answer
answer_agent = AnswerAgent()
answer_result = await answer_agent.execute((query, results, intent))

print("Answer:")
print(answer_result.output["answer"])
print("\nCitations:")
for c in answer_result.output["citations"]:
    print(f"- {c.title}")